## Bước 1: Gọi các thư viện hỗ trợ

In [1]:
import os
import time
import glob
import math
import json
import numpy as np 
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## Bước 2: Khởi tạo kích thước quần thể & số lượng biến 

In [2]:
def initialize_population(num_individual, num_variable, option, algorithm, random_seed): 
    """
    Hàm khởi tạo quần thể: 

    
    """
    np.random.seed(random_seed)
    if option == 'sphere' or option == 'rastrigin':
        bounds = np.asarray([(-5.12, 5.12) for i in range(num_variable)])
    if option == 'rosenbrock':
        bounds = np.asarray([(-5.0, 10.0) for i in range(num_variable)])
    if option == 'griewank':
        bounds = np.asarray([(-600.0, 600.0) for i in range(num_variable)])
    if option == 'ackley':
        bounds = np.asarray([(-32.768, 32.768) for i in range(num_variable)]) 

    if algorithm == 'de':
        pop = bounds[:, 0] + (np.random.rand(num_individual, len(bounds)) * (bounds[:, 1] - bounds[:, 0]))
    if algorithm == 'es':
        pop = bounds[:, 0] + (np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0]))
    return pop, bounds

## Bước 3: Chuẩn bị các hàm mục tiêu

### Hàm sphere function

<p align="center">
<img 
    src="https://www.sfu.ca/~ssurjano/spheref.png"
    width="400" 
    height="300"/>
</p>

<p align="center">
    <img src="https://latex.codecogs.com/svg.image?f(x)&space;=&space;\sum_{i&space;=&space;1}^{n}{x_{i}^{2}}" title="f(x) = \sum_{i = 1}^{n}{x_{i}^{2}}"/>
</p>

In [372]:
def sphere_function(ind): 
    """
    Hàm mục tiêu sphere_function

    Arguments: 
        ind: 
            Cho biết gen của cá thể ind: 
            Ví dụ: 
            ind = [1.0, 1.4, 6.4]

    Output: 
        Trả về một số nguyên dương sau khi được tính toán bởi hàm sphere_function
        Ví dụ: 
        1.31514134114    
    """
    result = 0
    for idx in range(0, len(ind)):
        result = result + (ind[idx] ** 2)
    return result

### Hàm rastrigin function

<p align="center">
<img 
    src="https://www.sfu.ca/~ssurjano/rastr.png"
    width="400" 
    height="300"/>
</p>

<p align="center">
    <img src="https://latex.codecogs.com/svg.image?f(x)&space;=&space;10d&space;&plus;&space;\sum_{i=1}^{d}{\[x_{i}^{2}&space;-&space;10cos(2\pi&space;x_{i})\]}" title="f(x) = 10d + \sum_{i=1}^{d}{\[x_{i}^{2} - 10cos(2\pi x_{i})\]}" />
</p>

In [373]:
def rastrigin_function(ind): 
    """ 
    Hàm mục tiêu rastrigin

    Arguments: 
        ind: 
            Cho biết gen của cá thể ind: 
            Ví dụ: 
            ind = [1.0, 1.4, 6.4]

    Output: 
        Trả về một số nguyên dương sau khi được tính toán bởi hàm rastrigin_function
        Ví dụ: 
        1.31514134114   
    """ 
    result = 0
    for idx in range(0, len(ind)):
        result += (ind[idx]**2 - 10*math.cos(2*math.pi*ind[idx]))
    return result + 10*len(ind)

### Hàm rosenbrock function

<p align="center">
<img 
    src="https://www.sfu.ca/~ssurjano/rosen.png"
    width="400" 
    height="300"/>
</p>

<p align="center">
    <img src="https://latex.codecogs.com/svg.image?f(x)&space;=&space;\sum_{i=1}^{d-1}{\[100(x_{i&plus;1}-x_{i}^{2})^{2}&space;&plus;&space;(x_{i}&space;-&space;1)^{2}&space;\]}" title="f(x) = \sum_{i=1}^{d-1}{\[100(x_{i+1}-x_{i}^{2})^{2} + (x_{i} - 1)^{2} \]}" />
</p>

In [374]:
def rosenbrock_function(ind): 
    """ 
    Hàm mục tiêu rosenbrock

    Arguments: 
        ind: 
            Cho biết gen của cá thể ind: 
            Ví dụ: 
            ind = [1.0, 1.4, 6.4]

    Output: 
        Trả về một số nguyên dương sau khi được tính toán bởi hàm rosenbrock_function
        Ví dụ: 
        1.31514134114   
    """ 
    result = 0
    for idx in range(0, len(ind) - 1):
        result += 100*(ind[idx + 1] - ind[idx]**2)**2 + (ind[idx] - 1)**2
    return result   

### Hàm Griewank function

<p align="center">
<img 
    src="https://www.sfu.ca/~ssurjano/griewank.png"
    width="400" 
    height="300"/>
</p>

<p align="center">
    <img src="https://latex.codecogs.com/svg.image?f(x)&space;=&space;\sum_{i=1}^{d}{\dfrac{x_{i}^{2}}{4000}&space;-&space;\prod_{i=1}^{d}{cos{\left(\dfrac{x_{i}}{\sqrt{i}}\right)}&space;&plus;&space;1}}" title="f(x) = \sum_{i=1}^{d}{\dfrac{x_{i}^{2}}{4000} - \prod_{i=1}^{d}{cos{\left(\dfrac{x_{i}}{\sqrt{i}}\right)} + 1}}" />
</p>

In [375]:
def griewank_function(ind): 
    """ 
    Hàm mục tiêu Griewank

    Arguments: 
        ind: 
            Cho biết gen của cá thể ind: 
            Ví dụ: 
            ind = [1.0, 1.4, 6.4]

    Output: 
        Trả về một số nguyên dương sau khi được tính toán bởi hàm griewank_function
        Ví dụ: 
        1.31514134114   
    """ 
    result_a = 0 
    result_b = 1
    for idx in range(0, len(ind)):
        result_a += (ind[idx]**2)/4000
    
    for idx in range(0, len(ind)): 
        result_b *= math.cos(ind[idx] / math.sqrt(idx + 1))

    return result_a - result_b + 1

### Hàm ackley function

<p align="center">
<img 
    src="https://www.sfu.ca/~ssurjano/ackley.png"
    width="400" 
    height="300"/>
</p>

<p align="center">
    <img src="https://latex.codecogs.com/svg.image?f(x)&space;=&space;-a&space;\exp{\left(-b&space;\sqrt{\dfrac{1}{d}&space;\sum_{i=1}^{d}{x_{i}^{2}}}&space;\right)}&space;-&space;\exp{\left(\dfrac{1}{d}&space;\sum_{i=1}^{d}{cos\left(cx_{i}\right)}&space;&space;\right)}&space;&plus;&space;a&plus;&space;\exp{\left(1\right)}" title="f(x) = -a \exp{\left(-b \sqrt{\dfrac{1}{d} \sum_{i=1}^{d}{x_{i}^{2}}} \right)} - \exp{\left(\dfrac{1}{d} \sum_{i=1}^{d}{cos\left(cx_{i}\right)} \right)} + a+ \exp{\left(1\right)}" />
</p>

In [376]:
def ackley_function(ind , a=20, b=0.2, c=2*math.pi): 
    """ 
    Hàm mục tiêu Ackley

    Arguments: 
        ind: 
            Cho biết gen của cá thể ind: 
            Ví dụ: 
            ind = [1.0, 1.4, 6.4]
        a: 
            Mặc định bằng 20. 
        b: 
            Mặc định bằng 0.2.  
        c: 
            Mặc định bằng 2*pi. 
            
    Output: 
        Trả về một số nguyên dương sau khi được tính toán bởi hàm ackley_function
        Ví dụ: 
        1.31514134114   
    """ 
    sum_1 = 0
    sum_2 = 0
    for idx in range(0, len(ind)):
        sum_1 += ind[idx]**2
        sum_2 += math.cos(c*ind[idx])
    term_1 = -a*math.exp(-b*math.sqrt(sum_1/len(ind)))
    term_2 = -math.exp(sum_2/len(ind))
    return term_1 + term_2 + a + math.exp(1)

### Hàm ước lượng độ thích nghi của cá thể

In [377]:
def evaluate_individual(ind, option):
    global accumulative_call
    accumulative_call += 1
    if option == 'sphere':
        result = sphere_function(ind)
    if option == 'rastrigin':
        result = rastrigin_function(ind)
    if option == 'rosenbrock':
        result = rosenbrock_function(ind)
    if option == 'griewank':
        result = griewank_function(ind)
    if option == 'ackley':
        result = ackley_function(ind)
    return result 

### Hàm ước lượng độ thích nghi của quần thể

In [378]:
def evaluate_population(pop, option, algorithm): 
    if algorithm == 'es':
        global accumulative_call
        accumulative_call += len(pop)
    if option == 'sphere':
        result = [sphere_function(ind) for ind in pop]
    if option == 'rastrigin':
        result = [rastrigin_function(ind) for ind in pop]
    if option == 'rosenbrock':
        result = [rosenbrock_function(ind) for ind in pop]
    if option == 'griewank':
        result = [griewank_function(ind) for ind in pop]
    if option == 'ackley':
        result = [ackley_function(ind) for ind in pop]
    return result

## Bước 4: Khởi tạo hàm lưu trữ thông tin

In [379]:
def tocsv(result, num_individual, num_variable, option, random_seed, algorithm):
    src = './data/{}/{}/{}/{}/'.format(algorithm, num_variable, option, num_individual)
    if os.path.exists(src) == False:
        os.makedirs(src)
    json_file = open(src+'{}.json'.format(random_seed), 'w')
    print('Storing to', src + '{}.json'.format(random_seed))
    json.dump(result, json_file, indent=4)

## Bước 5: Khởi tạo hàm trực quan hóa quá trình tiến hóa

In [380]:
def visualise(pop, algorithm, iteration, num_individual, option, seed):
    """
    Hàm visualse - Trực quan hóa quá trình di chuyển của các cá thể. 

    Arguments: 
        pop: 
            Cho biết vị trí mỗi các thể đang đứng tại đâu. 
        
        iteration: 
            Cho biết thế hệ của quần thể đang được trực quan hóa. 

        target_function: 
            Cho biết hàm mục tiêu đang được sử dụng là hàm nào.
    
    Output: 
        Hàm sẽ lưu một file ảnh *.png với địa chỉ sau: 
            './visualise/<Tên hàm mục tiêu>/<Kích thước quần thể>/iteration_<lần iteration bao nhiêu>.png"
        
        Ví dụ: 
            +visualse: 
                +sphere: 
                    +32: 
                        +iteration_0.png
                        +iteration_1.png
                        ...
                    +64: 
                        +iteration_0.png
                        ...
                +rastrigin: 
                    +32: 
                        +iteration_0.png
                        +iteration_1.png
    """
    fig = plt.figure(num=1, clear=True)
    ax = fig.add_subplot(1, 1)
    
    if option == 'sphere':
        (x, y) = np.meshgrid((np.arange(-8, 8, 0.01), np.arange(-8, 8, 0.01))
        z = sphere_function([x, y])
    if option == 'rastrigin':
        (x, y) = np.meshgrid((np.arange(-4, 4, 0.1), np.arange(-4, 4, 0.1))
        z = rastrigin_function([x, y])
    if option == 'rosenbrock':
        (x, y) = np.meshgrid((np.arange(-20.0, 20.0, 0.01), np.arange(-20.0, 20.0, 0.01))
        z = rosenbrock_function([x, y])
    if option == 'griewank':
        (x, y) = np.meshgrid((np.arange(-60, 60, 1), np.arange(-60, 60, 1))
        z = griewank_function([x, y])
    if option == 'ackley':
        (x, y) = np.meshgrid((np.arange(-60, 60, 1), np.arange(-60, 60, 1))
        z = ackley_function([x, y])
    
    X = [p[0] for p in pop]
    Y = [p[1] for p in pop]

        
    ax.scatter(X, Y)
    ax.contour(x, y, z, cmap=cm.magma, alpha=0.3)
    ax.set_title('{} {}'.format(option, iteration))
    ax.set_xlabel('feature_x')
    ax.set_ylabel('feature_y')
    
    if os.path.exists('./visualise/{}/{}/{}'.format(algorithm, option, num_individual)) == False:
        os.makedirs('./visualise/{}/{}/{}'.format(algorithm, option, num_individual))
    
    fig.savefig('./visualise/{}/{}/{}/{}_{}.png'.format(algorithm, option, num_individual, seed, iteration))
    plt.clf() 
    plt.cla()
    plt.close()

In [381]:
def get_gif(option, iteration, seed, num_individual):
    """
    Hàm chuyển đổi tập hình ảnh *.png sang *.gif

    Hàm sẽ đọc địa chỉ thư mục chứa thư mục cha của các ảnh sau đó lấy toàn bộ ảnh dựa
    theo chỉ số iteration từ bé đến lớn sau đó merge lại thành một file gif. 

    Arguments: 
        pop: 
            Cho biết kích thước quần thể. 
        
        target_function: 
            Cho biết thế hệ quần thể đang xét. 
     
    Output: 
        Trả về một file gif tên <Hàm mục tiêu>_<Kích thước quần thể>.gif ngay thư mục thực thi ipynb. 
    """
    frames = [] 
    for i in range(0, iteration): 
        new_frame = Image.open('./visualise/{}/{}/{}/{}_{}.png'.format(algorithm, option, num_individual, seed, i))
        frames.append(new_frame)
        
    frames[0].save('./visualise/{}/{}/{}_{}_{}_{}.gif'.format(option, algorithm, option, num_individual, seed), 
                       format='GIF', append_images=frames[1:], save_all=True, duration=300, loop=0)      

## Bước 6: Kiểm tra hội tụ của hàm DE

In [382]:
def convergence_de(pop_fitness): 
    eps = 1e-8
    upper = np.all(pop_fitness <= pop_fitness[0] + eps)
    lower = np.all(pop_fitness >= pop_fitness[0] - eps) 
    return upper and lower

## Bước 7: Kiểm tra hội tụ của hàm ES

In [383]:
def convergence_es(num_individual, result, logger):
    eps = 1e-8
    k_inds = int(math.log2(num_individual))
    if len(result) <= k_inds:
        return False
    k_loggers = logger[-k_inds:]
    fitness = [f for f in k_loggers]
    min_fit = min(fitness)
    max_fit = max(fitness)
    return max_fit - min_fit <= eps

## Bước 8: Điều kiện dừng hàm

In [384]:
def stop(num_variable, iteration):
    if num_variable == 2 and iteration == 100000:
        return True
    if num_variable == 10 and iteration == 1000000:
        return True
    return False

## Bước 9: Hàm round

In [385]:
def get_round(best_offspring):
    arr = []
    for i in best_offspring:
        arr.append(round(i,5))
    return arr

## Bước 10: Khởi tạo thuật toán Differential Evolution

In [386]:
def differential_evolution(pop, num_individual, num_variable, option, random_seed, algorithm, p=0.5, w=1):
    iteration = 0
    result = {}

    global accumulative_call
    accumulative_call = 0

    pop_fitness = evaluate_population(pop, option, algorithm)

    while convergence_de(pop_fitness) == False and stop(num_variable, iteration) == False:
        iteration += 1
        for k, ind in enumerate(pop): 
            candidates = [candidate for candidate in range(0, num_individual) if candidate != k]
            a, b, c = pop[np.random.choice(candidates, 3, replace=False)] 
            z = a + w*(b-c)
            j = np.random.randint(low=0, high=num_individual) 

            tmp_ind = [z[i] if (i == j or np.random.rand() < p) else ind[i] for i in range(0, len(ind))]

            if evaluate_individual(tmp_ind, option) < evaluate_individual(ind, option): 
                pop[k] = tmp_ind
                
        pop_fitness = evaluate_population(pop, option)
        index = pop_fitness.index(min(pop_fitness))
        best_offspring = pop[index].tolist()

        result['iteration {}'.format(iteration)] = {
            'best_individual' : get_round(best_offspring),
            'best_fitness' : round(pop_fitness[index], 5),
            'num of evaluation in current': accumulative_call
        }
        visualise(pop, algorithm, iteration, option, random_seed)
    return get_gif(option, iteration, random_seed, num_individual) 
    #return tocsv(result, num_individual, num_variable, option, random_seed, algorithm)

## Bước 11: Khởi tạo thuật toán Evoluation Strategies

In [387]:
def evolution_strategies(pop, num_individual, num_variable, option, bounds, random_seed, algorithm, c_inc=1.1, c_dec=0.6, step_size=1):
    iteration = 0
    logger = []
    result = {} 

    global accumulative_call
    accumulative_call = 0

    #Đánh giá độ thích nghi của quần thể cha me
    pop_fitness = evaluate_individual(pop, option)
    logger.append(pop_fitness)

    while convergence_es(num_individual, result, logger) == False and stop(num_variable, iteration) == False:
        iteration += 1

        #Tạo cá thể con
        epsilon = np.random.randn(num_individual, len(bounds))
        offspring = pop + step_size*epsilon
        offspring = np.clip(offspring, bounds[:,0], bounds[:,1])

        #Đánh giá cá thể con
        offspring_fitness = evaluate_population(offspring, option, algorithm)
        best_idx = offspring_fitness.index(min(offspring_fitness))
        best_fitness = offspring_fitness[best_idx]
        best_offspring = offspring[best_idx]
        
        #So sánh
        if best_fitness <= pop_fitness:
            pop = best_offspring.copy()
            pop_fitness = best_fitness
            step_size *= c_inc
        else:
            step_size *= c_dec

        logger.append(best_fitness)
        result['iteration {}'.format(iteration)] = {
            'best_individual' : get_round(pop.tolist()),
            'best_fitness' : round(best_fitness, 2),
            'num of evaluation in current': accumulative_call
        }
        visualise(offspring, algorithm, iteration, option, random_seed)
    return get_gif(option, iteration, random_seed, num_individual)     
    #return tocsv(result, num_individual, num_variable, option, random_seed, algorithm)

## Bước 12: Lựa chọn thuật toán để chạy

In [388]:
def run_all(num_individual, num_variable, option, algorithm, random_seeds):
    for random_seed in random_seeds:
        accumulative_call = 0 
        pop, bounds = initialize_population(num_individual, num_variable, option, algorithm, random_seed)
        if algorithm == 'de': 
            differential_evolution(pop, num_individual, num_variable, option, random_seed, algorithm)
        if algorithm == 'es':
            evolution_strategies(pop, num_individual, num_variable, option, bounds, random_seed, algorithm)
        #get_gif(pop, option)
        

In [ ]:
set_option = ['sphere', 'rastrigin', 'rosenbrock', 'griewank', 'ackley']
random_seeds = [(19521281 + i) for i in range(0, 1)]
num_individual = [32, 64, 128, 256, 512, 1024]
for opt in set_option:    
    for num in num_individual:
        run_all(num, 2, opt, 'es', random_seeds)
